In [1]:
import symforce
# symforce.set_epsilon_to_symbol()
symforce.set_epsilon_to_number()


# import numpy as np
# # import plotly.express as px
# import sympy as sm

# x = sm.Symbol("x")
# # eps = sm.Symbol("epsilon")
# epsilon = sm.Symbol("epsilon", positive=True)

In [2]:
import numpy as np
from symforce.values import Values
import symforce.symbolic as sf

# for optimisation
from symforce.opt.factor import Factor

from symforce.opt.optimizer import Optimizer

In [3]:

num_poses = 3
num_landmarks = 3


initial_values = Values(
    # populate this with initial poses 
    poses=[sf.Pose2.identity()] * num_poses,
    landmarks=[sf.V2(-2, 2), sf.V2(1, -3), sf.V2(5, 2)],
    distances=[1.7, 1.4],
    angles=np.deg2rad([[145, 335, 55], [185, 310, 70], [215, 310, 70]]).tolist(),
    epsilon=sf.numeric_epsilon,
)

In [4]:
def bearing_residual(
    pose: sf.Pose2, landmark: sf.V2, angle: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    t_body = pose.inverse() * landmark
    predicted_angle = sf.atan2(t_body[1], t_body[0], epsilon=epsilon)
    return sf.V1(sf.wrap_angle(predicted_angle - angle))

In [5]:
def odometry_residual(
    pose_a: sf.Pose2, pose_b: sf.Pose2, dist: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    return sf.V1((pose_b.t - pose_a.t).norm(epsilon=epsilon) - dist)

In [6]:

factors = []

# Bearing factors
for i in range(num_poses):
    for j in range(num_landmarks):
        factors.append(Factor(
            residual=bearing_residual,
            keys=[f"poses[{i}]", f"landmarks[{j}]", f"angles[{i}][{j}]", "epsilon"],
        ))

# Odometry factors
for i in range(num_poses - 1):
    factors.append(Factor(
        residual=odometry_residual,
        keys=[f"poses[{i}]", f"poses[{i + 1}]", f"distances[{i}]", "epsilon"],
    ))

In [7]:

optimizer = Optimizer(
    factors=factors,
    optimized_keys=[f"poses[{i}]" for i in range(num_poses)],
    # So that we save more information about each iteration, to visualize later:
    debug_stats=True,
    params=Optimizer.Params(verbose=True),  # Customize optimizer behavior
)

In [8]:
result = optimizer.optimize(initial_values)

RuntimeError: SYM_ASSERT: linearized_factor->index.tangent_dim == linearized_factor->jacobian.cols()
    --> void sym::Factor<ScalarType>::Linearize(const sym::Values<Scalar>&, sym::Factor<ScalarType>::LinearizedDenseFactor*, const std::vector<sym::index_entry_t>*) const [with ScalarType = double; sym::Factor<ScalarType>::LinearizedDenseFactor = sym::linearized_dense_factor_t]
    --> /project/symforce/opt/factor.cc:102


In [ ]:
from symforce.examples.robot_2d_localization.plotting import plot_solution
plot_solution(optimizer, result)